# Pré-Processamento

In [101]:
!pip install unidecode

/home/ruasgar/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/ruasgar/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.13.1-unknown is an invalid version and will not be supported in a future release
  warnings.warn(


In [1]:
import string
import re
import nltk
import ast
import pandas as pd
import numpy as np
import tensorflow as tf
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer
from collections import Counter
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin

nltk.download('stopwords')
nltk.download('punkt')

2023-06-13 18:31:37.569635: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-13 18:31:38.016019: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-13 18:31:38.017885: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 18:31:39.984265: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ruasgar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ruasgar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [103]:
df = pd.read_csv("../data/full_articles_dataset.csv",converters={'biased_words4':ast.literal_eval})

In [104]:
df.columns

Index(['sentence', 'news_link', 'outlet', 'topic', 'type', 'group_id',
       'num_sent', 'Label_bias', 'Label_opinion', 'article', 'biased_words4',
       'full_article'],
      dtype='object')

## 1 - Features Importantes
- Label_bias (categórica): Indica se o texto foi classificado como enviesado, não-enviesado ou se não foi possível atingir um consenso quanto a classificação.
- Biased_words(vetor): Indica as palavras marcadas como "denunciantes" da presença de viés.
- Topic(categórica): Indica o assunto do texto, dentro das categorias
- Type: Denota o posicionamento do viés, separados como esquerda, centro ou direita.
- Outlet: Portal de origem da notícia

In [105]:
df['Label_bias'].unique()

array(['Biased', 'Non-biased', 'No agreement'], dtype=object)

In [106]:
df['topic'].unique()

array(['elections-2020', 'sport', 'immigration', 'environment',
       'student-debt', 'vaccines', 'abortion', 'white-nationalism',
       'coronavirus', 'trump-presidency', 'middle-class', 'gender',
       'gun-control', 'international-politics-and-world-news'],
      dtype=object)

In [107]:
df['type'].unique()

array(['center', 'left', 'right'], dtype=object)

## 2 - Pré-Processamento

*Observação*: as funções relativas a entradas textuais são testadas individualmente com a coluna 'sentence'. Essa feature traz a frase na qual os participantes da pesquisa viram um viés.

Quando formos realizar os treinos e a validação, utilizaremos todo o texto que compõe o documento analisado. Se mostrar-se interessante, podemos averiguar a performance reduzindo o corpus a, justamente, essas sentenças. 

### 2.1 - Básico textual (remoção de stopwords, filtro de expressões com números, etc)

- A priori, vamos considerar como tokens todas as palavras que:
    - tenham mais do que 3 letras;
    - não possuam números;
    - não sejam siglas (como U.S.) nem que tenham um símbolo diferente de letra pós ou precedido por uma (como é o caso de separação entre palavras ao pular linha, e.g. "fari-");
    - não estejam nas stopwords do inglês.

- Uma decisão importante, especialmente no contexto de viés, é exclusão das das aspas. Isso pode ocorrer tanto de forma a ironizar acontecimentos como a corroborar algum conhecimento com argumento de autoridade enviesado, mas é algo que acreditamos estar diretamente relacionado ao conteúdo da frase, independetemente da grafia ao redor. Afinal, queremos tentar identificar a intenção. Seria interessante, para um próximo trabalho, tentar considerar as aspas e o que estivesse sob seus limites como algum tipo de incentivo ou demérito.

- Mediante a escolha do usuário, aplicaremos um stemmer(Porter/Lancaster) ou um lemmatizer. Também podemos simplesmente não aplicar nenhum.

In [108]:
digit_pattern = r'\d+(\.\d+)?'
solo_quotations_pattern = pattern = r"^(?:' '|\\" "|\`\`)$"

remove_quotation = r"'\b|\b'\s|\s'\b|\"\b|\b\"\s|\s\"\b|``\b|\b``\s|\s``\b" # como em "America is Great"
remove_symbols_only = r'\b[^\w\s]+\b' # como em '--'
remove_symbols_if_aside = r'\b[^\w\s]|[^\w\s]\b' # como em 'dog-'

remove_patterns = remove_quotation,remove_symbols_if_aside, remove_symbols_only

stop_words = set(stopwords.words('english')) # talvez o set deixe mais rápido

def preprocess_basic_text(data,columns):

    for col in columns:

       prepped_texts = []

       for doc in data[col]:

            if type(doc) == list:
              doc = ','.join(doc)
            # Remove acentos e põe tudo para lower case
            doc = unidecode(doc).lower()
            tokens = word_tokenize(doc,language="english")

            tokens = [
                re.sub('|'.join(remove_patterns),'',t)
                for t in tokens
                if not bool(re.search(digit_pattern,t))
                and t not in string.punctuation
                and t not in stop_words
            ]

            tokens = [
                t for t in tokens
                if not bool(re.match(solo_quotations_pattern,t))
                and len(t) >= 3
             ]

            prepped_texts.append(tokens)

       data[col] = prepped_texts

    return data

df_copy = df.copy()
df_copy = preprocess_basic_text(df_copy,['sentence'])
df_copy['sentence'].head()

0    [youtube, making, clear, birtherism, platform,...
1    [increasingly, bitter, dispute, american, wome...
2    [may, humanitarian, crisis, driving, vulnerabl...
3    [professor, teaches, climate, change, classes,...
4    [world, antidoping, agency, tuesday, said, rus...
Name: sentence, dtype: object

### 2.2 - Stemmers

- Temos o objetivo de avaliar como diferentes estratégias de stemming afetam os resultados do treinamento. A função a seguir visa abstrair esse passo para um etapa separada de pré-processamento.
- Serão testados os stemmer de Porter e de Lancaster, além da lematização via WordNet. Vamos também obter os resultados do modelo quando utilizados sem qualquer stemmer.

In [109]:
def apply_stemmer_individual(sentence, stemmer_type:str):

    # Função a ser executada para cada sentença, com expectativa de uso em algum filtro, por exemplo

    stemmer = {}
    lemmatizer = {}

    if stemmer_type == "Porter":
        stemmer = PorterStemmer()
        return [stemmer.stem(token) for token in sentence]

    elif stemmer_type == "Lancaster":
        stemmer = LancasterStemmer()
        return [stemmer.stem(token) for token in sentence]

    else:  # "Wordnet":
        lemmatizer = WordNetLemmatizer()
        return [lemmatizer.stem(token) for token in sentence]

In [110]:
df_copy['sentence'] = df_copy['sentence'].apply(lambda x: apply_stemmer_individual(x,"Porter"))
df_copy['sentence'].head()

0    [youtub, make, clear, birther, platform, year,...
1    [increasingli, bitter, disput, american, women...
2    [may, humanitarian, crisi, drive, vulner, peop...
3    [professor, teach, climat, chang, class, subje...
4    [world, antidop, agenc, tuesday, said, russian...
Name: sentence, dtype: object

In [111]:
# generalizando a aplicação pra todas as instâncias
def apply_stemmer_for_all(data,columns,stemmer_flag=False,stemmer_type=None):

  if stemmer_flag == False:
    return data

  for col in columns:
    data[col] = data[col].apply(lambda x: apply_stemmer_individual(x,stemmer_type))
  return data

### 2.3 - Encoding (palavras)

- Para representar as palavras, vamos criar um encoding com base na frequência apresentada no corpus.
- Utilizaremos alguns números para representar determinadas semânticas:
  - padding("pad"), necessário para deixar todas as entradas com mesmo tamanho, terá o valor 0.
  - start-of-sequence("sos"), usado para identificar o ínicio de cada sentença, terá o número 1
  - unknown, para marcar palavras desconhecidas, terá o número 2

In [112]:
SOS = "<sos>"
PADDING = "<pad>"
UNKNOWN = "<ukn>"

In [113]:
def add_padding_and_sos(data, columns):


    for col in columns:
        max_length = max(len(seq) for seq in data[col])
        for index,row in data.iterrows():
            document = [SOS] + row[col]
            paddings = (max_length - len(row[col])) * [PADDING]
            document += paddings
            data.at[index,col] = document

    return data

In [114]:
df_copy = add_padding_and_sos(df_copy, ['sentence'])

In [115]:
print(df_copy.loc[190,'sentence'])

['<sos>', 'even', 'though', 'thunberg', 'capabl', 'thing', 'mani', 'right', 'refus', 'read', 'understand', 'scientif', 'evid', 'rightwing', 'argu', 'thunberg', 'fellow', 'youth', 'activist', 'think', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


- Construindo o vocabulário com base na frequência de aparecimento.
  - *Obs*: note a colocação de determinadas tags logo no ínicio

In [116]:
def create_vocabulary(data, columns):
  """ Retorna uma lookup table (StaticVocabularyTable) """

  vocab = Counter()
  for col in columns:
      for document in data[col]:
          vocab.update(document)

  words = tf.constant([PADDING] +
                      [SOS] +
                      [UNKNOWN] +
                      [word for word in vocab.keys() if word != PADDING and word != SOS]
  )
  words_ids = tf.range(len(vocab)+1, dtype=tf.int64) # +1 porque unknown é o único não mapeado
  vocab_init = tf.lookup.KeyValueTensorInitializer(words,words_ids)
  num_oov_buckets = 5000 # bucket para palavras desconhecidas
  vocab_table = tf.lookup.StaticVocabularyTable(vocab_init,num_oov_buckets)

  return vocab_table

- Demonstração de como estão os encodings de algumas palavras.

In [117]:
vocab_table = create_vocabulary(df_copy, columns=['sentence'])
vocab_table.lookup(tf.constant([b'nice trump move'.split(b' ')]))

<tf.Tensor: shape=(1, 3), dtype=int64, numpy=array([[2390,   97,  558]])>

- Convertendo os tokens para o índice na tabela.

In [118]:
def convert_words_to_freq(data, vocab, columns):

  for index, row in data.iterrows():

    for col in columns:
      doc = tf.constant(row[col])
      doc_ided = vocab_table.lookup(doc).numpy() if len (doc) > 0 else []
      data.at[index, col] = doc_ided

  return data

In [119]:
df_copy = convert_words_to_freq(df_copy, vocab_table, ['sentence'])
df_copy.head(2)

,sentence,news_link,outlet,topic,type,group_id,num_sent,Label_bias,Label_opinion,article,biased_words4,full_article
0,"[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",https://eu.usatoday.com/story/tech/2020/02/03/...,usa-today,elections-2020,center,1,1,Biased,Somewhat factual but also opinionated,YouTube says no ‘deepfakes’ or ‘birther’ video...,"[belated, birtherism]",('YouTube is making clear there will be no “bi...
1,"[1, 18, 19, 20, 21, 22, 23, 24, 25, 24, 26, 27...",https://www.nbcnews.com/news/sports/women-s-te...,msnbc,sport,left,1,1,Non-biased,Entirely factual,"FRISCO, Texas — The increasingly bitter disput...",[bitter],"('', 'Profile', 'Sections', 'tv', 'Featured', ..."


### 2.4 - One-hot encoding para features categóricas

In [120]:
def cats_to_one_hot(data,column_names):
  for col in column_names:
    df_encoded = pd.get_dummies(df[col], prefix=col)
    cat_col_index = data.columns.get_loc(col)
    data = pd.concat([data.iloc[:,:cat_col_index], df_encoded, data.iloc[:,cat_col_index+1:]],axis=1)
  return data

In [121]:
df_copy = cats_to_one_hot(df_copy, column_names=['topic','outlet','type','Label_opinion'])
df_copy.head(3)

,sentence,news_link,outlet_alternet,outlet_breitbart,outlet_federalist,outlet_fox-news,outlet_huffpost,outlet_msnbc,outlet_reuters,outlet_usa-today,...,group_id,num_sent,Label_bias,Label_opinion_Entirely factual,Label_opinion_Expresses writer’s opinion,Label_opinion_No agreement,Label_opinion_Somewhat factual but also opinionated,article,biased_words4,full_article
0,"[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",https://eu.usatoday.com/story/tech/2020/02/03/...,False,False,False,False,False,False,False,True,...,1,1,Biased,False,False,False,True,YouTube says no ‘deepfakes’ or ‘birther’ video...,"[belated, birtherism]",('YouTube is making clear there will be no “bi...
1,"[1, 18, 19, 20, 21, 22, 23, 24, 25, 24, 26, 27...",https://www.nbcnews.com/news/sports/women-s-te...,False,False,False,False,False,True,False,False,...,1,1,Non-biased,True,False,False,False,"FRISCO, Texas — The increasingly bitter disput...",[bitter],"('', 'Profile', 'Sections', 'tv', 'Featured', ..."
2,"[1, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51...",https://www.alternet.org/2019/01/here-are-5-of...,True,False,False,False,False,False,False,False,...,1,1,Biased,False,True,False,False,Speaking to the country for the first time fro...,[crisis],('Speaking to the country for the first time f...


### 2.5 - Limpando features

In [122]:
def drop_columns(data, column_names):
  return data.drop(column_names,axis=1)

In [123]:
df_copy = drop_columns(df_copy,column_names=['news_link','group_id','article'])
df_copy.head(1)
df_copy.shape

(1414, 34)

### 2.6 - Compilando pipeline de pré-processamento

In [124]:
# criando vocabulário separadamente para ter acesso a seu tamanho
#vocab = create_vocabulary(df.copy(),['full_article'])

In [125]:
class VocabularyConversionTransformer(BaseEstimator,TransformerMixin):
  def __init__(self,vocab_columns, columns_to_be_converted):
    self.vocab_table = None
    self.columns_to_be_converted = columns_to_be_converted
    self.vocab_columns = vocab_columns

  def fit(self, X, y=None):
    self.vocab_table = create_vocabulary(X,self.vocab_columns)
    return self

  def transform(self,X,y=None):
    return convert_words_to_freq(X,columns=self.columns_to_be_converted,vocab=self.vocab_table)
    
  def get_vocabulary(self):
    return self.vocab_table
  

In [133]:
def preprocess(data,stemmer_flag=False,stemmer_type=None):

  one_hot_columns = ['topic','outlet','type']
  columns_to_drop = ['news_link','group_id', 'Label_opinion']
  text_columns = ['full_article']

  preprocess_pipeline = [
      ('Tokenização', FunctionTransformer(preprocess_basic_text,kw_args={'columns': text_columns})),
      ('Stemming', FunctionTransformer(apply_stemmer_for_all, kw_args={'columns':text_columns,
                                                                      'stemmer_flag': stemmer_flag,
                                                                      'stemmer_type': stemmer_type
                                                                      })),
      ('SOS/Padding Addition', FunctionTransformer(add_padding_and_sos, kw_args={'columns':text_columns})),
      ('Vocab Creation and Word-Int Conversion', VocabularyConversionTransformer(vocab_columns=text_columns,columns_to_be_converted=text_columns)),
      ('One-hot-encoding', FunctionTransformer(cats_to_one_hot,kw_args={'column_names':one_hot_columns})),
      ('Dropping features', FunctionTransformer(drop_columns,kw_args={'column_names':columns_to_drop})),
  ]

  pipeline = Pipeline(preprocess_pipeline)

  data = pipeline.fit_transform(data)

  vocab = pipeline.named_steps['Vocab Creation and Word-Int Conversion'].get_vocabulary()

  return data, vocab

In [129]:
data, vocab = preprocess(df.copy(),True,stemmer_type="Porter")

In [130]:
data.shape

(1414, 35)

In [131]:
data.columns

Index(['sentence', 'outlet_alternet', 'outlet_breitbart', 'outlet_federalist',
       'outlet_fox-news', 'outlet_huffpost', 'outlet_msnbc', 'outlet_reuters',
       'outlet_usa-today', 'topic_abortion', 'topic_coronavirus',
       'topic_elections-2020', 'topic_environment', 'topic_gender',
       'topic_gun-control', 'topic_immigration',
       'topic_international-politics-and-world-news', 'topic_middle-class',
       'topic_sport', 'topic_student-debt', 'topic_trump-presidency',
       'topic_vaccines', 'topic_white-nationalism', 'type_center', 'type_left',
       'type_right', 'num_sent', 'Label_bias',
       'Label_opinion_Entirely factual',
       'Label_opinion_Expresses writer’s opinion',
       'Label_opinion_No agreement',
       'Label_opinion_Somewhat factual but also opinionated', 'article',
       'biased_words4', 'full_article'],
      dtype='object')

### 2.7 - Salvando dataset final


In [134]:
data.to_csv('../data/final_dataset.csv')